HACKATON; CENICAFE 

CAFETERITOS 

🧪 Análisis de Datos Eddy Covariance para Modelado de NEE

Objetivo: Entrenar un modelo de machine learning para estimar el Intercambio Neto de CO₂ (NEE) en base a variables meteorológicas y de teledetección, usando datos de torre Eddy Covariance.


In [ ]:
# ====================================
# MODELO DE MACHINE LEARNING PARA NEE
# ====================================
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt

In [ ]:
# Instalar paquetes necesarios
%pip install pandas matplotlib seaborn scikit-learn xgboost scipy statsmodels geemap

import seaborn as sns

1. 📂 Carga y limpieza de datos

Se cargan los datos desde un archivo .csv separado por tabulaciones (\t). Los valores faltantes vienen codificados como -9999, por lo que se reemplazan por NaN para su correcto tratamiento.

También se convierte la columna TIMESTAMP_START al formato de fecha y hora estándar (datetime) para facilitar el filtrado temporal.

In [ ]:
# ---------------------------
# 1. Cargar los datos
# ---------------------------
archivo = "AMF_CR-Fsc_BASE_HH_2-5.csv"  # ← CAMBIA este nombre por el archivo real

# Leer el archivo CSV eliminando las primeras 2 filas de metadatos
df = pd.read_csv(archivo, sep=",", skiprows=2, low_memory=False)

print(f"✅ Archivo leído exitosamente")

print(f"📊 Filas: {len(df)}")
print(f"📋 Columnas: {len(df.columns)}")
print(f"📋 Primeras 5 columnas: {list(df.columns[:5])}")
print(f"\n📋 Primeras 3 filas:")
print(df.head(3))

2. ⏳ Filtro temporal: desde 9 de diciembre de 2016 a las 10:30

Los datos de teledetección (NDVI, PRI) solo están disponibles desde finales de 2016. Por tanto, se filtran los registros para trabajar solo con los datos a partir del 9 de diciembre de 2016 a las 10:30 am.

3. 🕒 Extracción de variables temporales

Se derivan columnas auxiliares a partir del timestamp para capturar la estacionalidad (doy: día del año) y el ciclo diurno (hour: hora decimal).

In [ ]:
# ---------------------------
# 2. Limpieza inicial
# ---------------------------
df.replace(-9999, np.nan, inplace=True)
df['TIMESTAMP_START'] = pd.to_datetime(df['TIMESTAMP_START'], format="%Y%m%d%H%M")

# ---------------------------
# 3. Filtro desde NDVI no NaN
# ---------------------------
# Filtrar registros donde NDVI no sea NaN
df = df[df['NDVI'].notna()].reset_index(drop=True)

print("Tamaño de la base tras filtro NDVI no NaN:", len(df))
print("Periodo de la base de datos después del filtro NDVI no NaN:")
print("Inicio:", df['TIMESTAMP_START'].min())
print("Fin   :", df['TIMESTAMP_START'].max())

# Analizar huecos temporales en la serie cada 30 minutos
df_sorted = df.sort_values('TIMESTAMP_START').reset_index(drop=True)
intervalo_esperado = pd.Timedelta(minutes=30)
diferencias = df_sorted['TIMESTAMP_START'].diff()
huecos = df_sorted[diferencias > intervalo_esperado]

print(f"Total de huecos encontrados: {len(huecos)}")
if not huecos.empty:
    print("Primeros 5 huecos:")
    for idx in huecos.index[:5]:
        anterior = df_sorted.loc[idx - 1, 'TIMESTAMP_START']
        actual = df_sorted.loc[idx, 'TIMESTAMP_START']
        print(f"Hueco entre {anterior} y {actual} (diferencia: {actual - anterior})")
else:
    print("No se encontraron huecos temporales mayores a 30 minutos.")

4. 🔢 Selección de variables predictoras y objetivo

A continuación se listan las variables seleccionadas como entradas del modelo y su variable objetivo. Se incluye su unidad y una breve descripción: 

| Variable       | Unidad              | Descripción                                                                 |
|----------------|----------------------|-----------------------------------------------------------------------------|
| `TA_1_1_1`     | °C                   | Temperatura del aire (sensor a 1.1 m)                                       |
| `RH_1_1_1`     | %                    | Humedad relativa (sensor a 1.1 m)                                           |
| `VPD_PI`       | hPa                  | Déficit de presión de vapor                                                |
| `SW_IN_1_1_1`  | W m⁻²                | Radiación solar de onda corta entrante                                     |
| `LW_IN`        | W m⁻²                | Radiación de onda larga entrante                                           |
| `P`            | mm                   | Precipitación acumulada                                                    |
| `WS`           | m s⁻¹                | Velocidad del viento                                                       |
| `NDVI`         | adimensional (0–1)   | Índice de vegetación normalizado (NDVI)                                    |
| `PRI`          | adimensional         | Índice de reflectancia fotosintética (Photochemical Reflectance Index)     |
| `doy`          | Día del año (1–366)  | Día del año para capturar estacionalidad                                   |
| `hour`         | Hora decimal (0–24)  | Hora decimal del día para capturar el ciclo diurno                         |
| `NEE_PI_F`     | toneladas CO₂/ha     | **Intercambio Neto de Carbono (variable objetivo a predecir)**             |

In [ ]:
# ---------------------------
# 4. Variables temporales útiles
# ---------------------------
df['year'] = df['TIMESTAMP_START'].dt.year
df['month'] = df['TIMESTAMP_START'].dt.month
df['doy'] = df['TIMESTAMP_START'].dt.dayofyear
df['hour'] = df['TIMESTAMP_START'].dt.hour + df['TIMESTAMP_START'].dt.minute / 60






5. 📊 Análisis exploratorio de variables

Se presentan dos tipos de análisis:
	•	Estadísticas descriptivas de las variables predictoras (media, desviación, mínimo, máximo, etc.).
	•	Correlación entre todas las variables, enfocándose en la relación entre las entradas y el NEE (NEE_PI_F), usando un mapa de calor (heatmap).

Este análisis permite identificar relaciones clave, redundancias y la importancia potencial de cada variable.

In [ ]:
# ---------------------------
# 5. Variables seleccionadas
# ---------------------------
variables_entrada = [
    'TA_1_1_1', 'RH_1_1_1', 'VPD_PI',
    'SW_IN_1_1_1', 'LW_IN', 'P', 'WS',
    'NDVI', 'PRI', 'doy', 'hour'
]
variable_objetivo = 'NEE_PI_F'

In [ ]:
# ========================================
# ANÁLISIS PROFUNDO DE VARIABLES
# ========================================

# Filtrar datos válidos para el análisis
df_analisis = df.dropna(subset=variables_entrada + [variable_objetivo])

print(f"📊 RESUMEN GENERAL DEL DATASET")
print(f"{'='*50}")
print(f"Total de registros válidos: {len(df_analisis):,}")
print(f"Período de datos: {df_analisis['TIMESTAMP_START'].min()} - {df_analisis['TIMESTAMP_START'].max()}")
print(f"Años disponibles: {sorted(df_analisis['year'].unique())}")
print(f"Registros por año: {df_analisis['year'].value_counts().sort_index().to_dict()}")

# Crear figura con subplots
fig, axes = plt.subplots(4, 3, figsize=(20, 24))
fig.suptitle('📈 ANÁLISIS TEMPORAL DE VARIABLES DE ENTRADA Y OBJETIVO', fontsize=16, fontweight='bold')

# Análisis de cada variable
for i, var in enumerate(variables_entrada + [variable_objetivo]):
    row = i // 3
    col = i % 3
    ax = axes[row, col]
    
    # Serie temporal mensual
    monthly_data = df_analisis.groupby(df_analisis['TIMESTAMP_START'].dt.to_period('M'))[var].mean()
    monthly_data.plot(ax=ax, linewidth=2, color='steelblue')
    
    ax.set_title(f'{var}', fontsize=12, fontweight='bold')
    ax.set_xlabel('Tiempo')
    if var == variable_objetivo:
        ax.set_ylabel('toneladas CO₂/ha')
    else:
        ax.set_ylabel('Valor')
    ax.grid(True, alpha=0.3)
    ax.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

# Análisis estadístico detallado
print(f"\n📊 ESTADÍSTICAS DESCRIPTIVAS DETALLADAS")
print(f"{'='*60}")
stats = df_analisis[variables_entrada + [variable_objetivo]].describe()
print(stats.round(3))

# Análisis de valores faltantes
print(f"\n🔍 ANÁLISIS DE VALORES FALTANTES")
print(f"{'='*40}")
missing_data = df[variables_entrada + [variable_objetivo]].isnull().sum()
missing_pct = (missing_data / len(df) * 100).round(2)
missing_df = pd.DataFrame({
    'Variable': missing_data.index,
    'Valores_Faltantes': missing_data.values,
    'Porcentaje': missing_pct.values
}).sort_values('Porcentaje', ascending=False)
print(missing_df.to_string(index=False))

In [ ]:
# ========================================
# ANÁLISIS DE PATRONES TEMPORALES
# ========================================

# Crear figura para análisis de patrones temporales
fig, axes = plt.subplots(2, 2, figsize=(18, 12))
fig.suptitle('⏰ PATRONES TEMPORALES DE NEE Y VARIABLES CLAVE', fontsize=16, fontweight='bold')

# 1. Patrón diario promedio del NEE
ax1 = axes[0, 0]
daily_pattern = df_analisis.groupby('hour')[variable_objetivo].mean()
daily_pattern.plot(kind='line', ax=ax1, color='darkgreen', linewidth=3)
ax1.set_title('📅 Patrón Diario Promedio del NEE', fontsize=12, fontweight='bold')
ax1.set_xlabel('Hora del día')
ax1.set_ylabel('NEE (toneladas CO₂/ha)')
ax1.grid(True, alpha=0.3)
ax1.axhline(y=0, color='red', linestyle='--', alpha=0.7)

# 2. Patrón estacional del NEE
ax2 = axes[0, 1]
seasonal_pattern = df_analisis.groupby('doy')[variable_objetivo].mean()
seasonal_pattern.plot(kind='line', ax=ax2, color='darkblue', linewidth=2)
ax2.set_title('🌱 Patrón Estacional del NEE', fontsize=12, fontweight='bold')
ax2.set_xlabel('Día del año')
ax2.set_ylabel('NEE (toneladas CO₂/ha)')
ax2.grid(True, alpha=0.3)
ax2.axhline(y=0, color='red', linestyle='--', alpha=0.7)

# 3. Relación NEE vs Radiación Solar por hora
ax3 = axes[1, 0]
scatter = ax3.scatter(df_analisis['SW_IN_1_1_1'], df_analisis[variable_objetivo], 
                     c=df_analisis['hour'], cmap='viridis', alpha=0.6)
ax3.set_title('☀️ NEE vs Radiación Solar (coloreado por hora)', fontsize=12, fontweight='bold')
ax3.set_xlabel('Radiación Solar (W m⁻²)')
ax3.set_ylabel('NEE (toneladas CO₂/ha)')
ax3.grid(True, alpha=0.3)
ax3.axhline(y=0, color='red', linestyle='--', alpha=0.7)
plt.colorbar(scatter, ax=ax3, label='Hora del día')

# 4. Relación NEE vs Temperatura por estación
ax4 = axes[1, 1]
# Definir estaciones
df_analisis['season'] = df_analisis['month'].map({
    12: 'Verano', 1: 'Verano', 2: 'Verano',
    3: 'Otoño', 4: 'Otoño', 5: 'Otoño',
    6: 'Invierno', 7: 'Invierno', 8: 'Invierno',
    9: 'Primavera', 10: 'Primavera', 11: 'Primavera'
})

colors = {'Verano': 'red', 'Otoño': 'orange', 'Invierno': 'blue', 'Primavera': 'green'}
for season in df_analisis['season'].unique():
    season_data = df_analisis[df_analisis['season'] == season]
    ax4.scatter(season_data['TA_1_1_1'], season_data[variable_objetivo], 
               c=colors[season], alpha=0.6, label=season)

ax4.set_title('🌡️ NEE vs Temperatura por Estación', fontsize=12, fontweight='bold')
ax4.set_xlabel('Temperatura (°C)')
ax4.set_ylabel('NEE (toneladas CO₂/ha)')
ax4.grid(True, alpha=0.3)
ax4.legend()
ax4.axhline(y=0, color='red', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()

# Análisis de correlaciones por período del día
print(f"\n🔍 ANÁLISIS DE CORRELACIONES POR PERÍODO DEL DÍA")
print(f"{'='*60}")

# Definir períodos del día
df_analisis['periodo_dia'] = pd.cut(df_analisis['hour'], 
                                   bins=[0, 6, 12, 18, 24], 
                                   labels=['Madrugada', 'Mañana', 'Tarde', 'Noche'])

for periodo in df_analisis['periodo_dia'].unique():
    if pd.notna(periodo):
        periodo_data = df_analisis[df_analisis['periodo_dia'] == periodo]
        correlaciones_periodo = periodo_data[variables_entrada + [variable_objetivo]].corr()
        nee_corr = correlaciones_periodo[variable_objetivo].drop(variable_objetivo).sort_values(key=abs, ascending=False)
        
        print(f"\n📊 {periodo} ({len(periodo_data)} registros):")
        print(f"Top 3 variables más correlacionadas con NEE:")
        for i, (var, corr) in enumerate(nee_corr.head(3).items()):
            print(f"  {i+1}. {var}: {corr:.3f}")

# Análisis estacional
print(f"\n🌍 ANÁLISIS DE CORRELACIONES POR ESTACIÓN")
print(f"{'='*50}")

for season in df_analisis['season'].unique():
    season_data = df_analisis[df_analisis['season'] == season]
    correlaciones_season = season_data[variables_entrada + [variable_objetivo]].corr()
    nee_corr = correlaciones_season[variable_objetivo].drop(variable_objetivo).sort_values(key=abs, ascending=False)
    
    print(f"\n🌱 {season} ({len(season_data)} registros):")
    print(f"Top 3 variables más correlacionadas con NEE:")
    for i, (var, corr) in enumerate(nee_corr.head(3).items()):
        print(f"  {i+1}. {var}: {corr:.3f}")

In [ ]:
# ========================================
# ANÁLISIS DE DISTRIBUCIONES Y OUTLIERS
# ========================================

# Crear figura para análisis de distribuciones
fig, axes = plt.subplots(4, 3, figsize=(20, 16))
fig.suptitle('📊 DISTRIBUCIONES DE VARIABLES Y DETECCIÓN DE OUTLIERS', fontsize=16, fontweight='bold')

# Análisis de distribuciones y outliers
outlier_summary = []

for i, var in enumerate(variables_entrada + [variable_objetivo]):
    row = i // 3
    col = i % 3
    ax = axes[row, col]
    
    # Obtener datos de la variable
    data = df_analisis[var].dropna()
    
    # Crear histograma
    ax.hist(data, bins=50, alpha=0.7, color='skyblue', edgecolor='black', density=True)
    ax.set_title(f'{var}', fontsize=10, fontweight='bold')
    ax.set_xlabel('Valor')
    if var == variable_objetivo:
        ax.set_ylabel('toneladas CO₂/ha')
    else:
        ax.set_ylabel('Densidad')
    ax.grid(True, alpha=0.3)
    
    # Calcular estadísticas de outliers usando IQR
    Q1 = data.quantile(0.25)
    Q3 = data.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = data[(data < lower_bound) | (data > upper_bound)]
    outlier_pct = (len(outliers) / len(data)) * 100
    
    # Agregar líneas verticales para límites de outliers
    ax.axvline(lower_bound, color='red', linestyle='--', alpha=0.8, linewidth=2, label='Límite inferior')
    ax.axvline(upper_bound, color='red', linestyle='--', alpha=0.8, linewidth=2, label='Límite superior')
    
    # Agregar línea para la mediana
    ax.axvline(data.median(), color='green', linestyle='-', alpha=0.8, linewidth=2, label='Mediana')
    
    # Agregar leyenda solo en el primer subplot
    if i == 0:
        ax.legend(fontsize=8)
    
    # Guardar información de outliers
    outlier_summary.append({
        'Variable': var,
        'Total_Outliers': len(outliers),
        'Porcentaje_Outliers': round(outlier_pct, 2),
        'Limite_Inferior': round(lower_bound, 3),
        'Limite_Superior': round(upper_bound, 3),
        'Min': round(data.min(), 3),
        'Max': round(data.max(), 3),
        'Media': round(data.mean(), 3),
        'Mediana': round(data.median(), 3),
        'Desviacion_Std': round(data.std(), 3)
    })

# Remover subplots vacíos si los hay
for i in range(len(variables_entrada + [variable_objetivo]), 12):
    row = i // 3
    col = i % 3
    if row < 4 and col < 3:
        axes[row, col].remove()

plt.tight_layout()
plt.show()

# Mostrar resumen de outliers
print(f"\n🚨 RESUMEN DE OUTLIERS Y ESTADÍSTICAS")
print(f"{'='*80}")
outlier_df = pd.DataFrame(outlier_summary)
print(outlier_df.to_string(index=False))

# Análisis de valores extremos en NEE
print(f"\n⚠️  ANÁLISIS DE VALORES EXTREMOS EN NEE")
print(f"{'='*40}")
nee_data = df_analisis[variable_objetivo]
print(f"Valores más negativos (mayor absorción de CO₂):")
print(nee_data.nsmallest(5).to_string())
print(f"\nValores más positivos (mayor emisión de CO₂):")
print(nee_data.nlargest(5).to_string())

# Análisis de condiciones extremas
print(f"\n🌡️  CONDICIONES DURANTE VALORES EXTREMOS DE NEE")
print(f"{'='*50}")

# Condiciones durante máxima absorción
max_absorption_idx = nee_data.idxmin()
max_absorption_conditions = df_analisis.loc[max_absorption_idx, variables_entrada + ['TIMESTAMP_START']]
print(f"Condiciones durante máxima absorción (NEE = {nee_data.min():.3f}):")
print(f"Fecha: {max_absorption_conditions['TIMESTAMP_START']}")
for var in variables_entrada:
    print(f"  {var}: {max_absorption_conditions[var]:.3f}")

# Condiciones durante máxima emisión
max_emission_idx = nee_data.idxmax()
max_emission_conditions = df_analisis.loc[max_emission_idx, variables_entrada + ['TIMESTAMP_START']]
print(f"\nCondiciones durante máxima emisión (NEE = {nee_data.max():.3f}):")
print(f"Fecha: {max_emission_conditions['TIMESTAMP_START']}")
for var in variables_entrada:
    print(f"  {var}: {max_emission_conditions[var]:.3f}")

# Crear boxplot para comparar variables
fig, ax = plt.subplots(1, 1, figsize=(15, 8))
data_normalized = df_analisis[variables_entrada + [variable_objetivo]].copy()
# Normalizar para comparar en el mismo gráfico
from sklearn.preprocessing import StandardScaler
scaler_viz = StandardScaler()
data_normalized[variables_entrada + [variable_objetivo]] = scaler_viz.fit_transform(data_normalized[variables_entrada + [variable_objetivo]])

data_normalized.boxplot(ax=ax, rot=45, grid=True)
ax.set_title('📦 COMPARACIÓN DE DISTRIBUCIONES (DATOS NORMALIZADOS)', fontsize=14, fontweight='bold')
ax.set_ylabel('Valores normalizados (Z-score)')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

6. ⚙️ Preparación de los datos para entrenamiento

Se dividen los datos en dos subconjuntos:
	•	Entrenamiento: registros desde diciembre de 2016 hasta finales de 2017.
	•	Prueba: registros del año 2018, para evaluar la capacidad de generalización del modelo.

Adicionalmente, se aplica una normalización tipo Z-score (media cero, desviación unitaria) a las variables predictoras.

In [ ]:
# ---------------------------
# 6. Filtrar datos válidos y robustecer preprocesamiento
# ---------------------------

def eliminar_outliers_iqr(df, variables):
    df_filtrado = df.copy()
    for var in variables:
        Q1 = df_filtrado[var].quantile(0.25)
        Q3 = df_filtrado[var].quantile(0.75)
        IQR = Q3 - Q1
        lower = Q1 - 1.5 * IQR
        upper = Q3 + 1.5 * IQR
        df_filtrado = df_filtrado[(df_filtrado[var] >= lower) & (df_filtrado[var] <= upper)]
    return df_filtrado

# 1. Eliminar outliers en variables de entrada y objetivo
df_sin_outliers = eliminar_outliers_iqr(df, variables_entrada + [variable_objetivo])

# 2. Eliminar filas con valores faltantes en variables seleccionadas
df_modelo = df_sin_outliers.dropna(subset=variables_entrada + [variable_objetivo])

# 3. (Ya no se filtra por periodo temporal aquí)

print(f"Registros finales para modelado: {len(df_modelo):,}")
print(f"Ventana temporal: {df_modelo['TIMESTAMP_START'].min()} - {df_modelo['TIMESTAMP_START'].max()}")
print(f"Años incluidos: {sorted(df_modelo['year'].unique())}")

El paso de ~13,000 registros a 6,497 se debe a la combinación de varios filtros estrictos aplicados en el preprocesamiento, especialmente:

1. **Eliminación de outliers**:  
   Se usa la función `eliminar_outliers_iqr` para cada variable de entrada y la variable objetivo. Este método elimina cualquier fila donde alguna de estas variables esté fuera del rango intercuartílico (IQR). Si una fila es outlier en cualquier variable, se elimina. Esto ya puede reducir bastante el tamaño.

2. **Eliminación de filas con NaN en cualquier variable relevante**:  
   Después de quitar outliers, se eliminan todas las filas que tengan al menos un valor faltante (`NaN`) en cualquiera de las variables de entrada o en la variable objetivo.  
   - Si tienes 11 variables de entrada + 1 objetivo = 12 columnas, y los NaN están distribuidos (no siempre en la misma fila), la intersección de filas completas puede ser mucho menor que el total de filas con algún NaN individual.
   - Por ejemplo, si cada columna tiene un 10% de NaN, pero en diferentes filas, el resultado final puede ser mucho menos del 90% de los datos originales.

3. **Filtros previos**:  
   - Se eliminan filas con NDVI NaN al inicio.
   - Se filtra el periodo temporal (desde diciembre 2016).
   - Se reemplazan -9999 por NaN.

**En resumen:**  
La reducción drástica ocurre porque:
- Los NaN están distribuidos en distintas columnas y filas, así que al exigir que todas las variables estén presentes en la misma fila, solo quedan las filas donde absolutamente todas las variables relevantes tienen datos observados y no son outlier.
- Cada filtro (outlier, NaN, periodo, NDVI) va reduciendo el tamaño, y el efecto acumulativo es fuerte.


In [ ]:
# ---------------------------
# 7. Análisis de variables de entrada y objetivo
# ---------------------------

print("\n📊 Estadísticas de variables de entrada:\n")
print(df_modelo[variables_entrada].describe().transpose())

print("\n📈 Correlación con la variable objetivo (NEE_PI_F):\n")
correlaciones = df_modelo[variables_entrada + [variable_objetivo]].corr()
print(correlaciones[[variable_objetivo]].sort_values(by=variable_objetivo, ascending=False))

# Mapa de calor de correlación
plt.figure(figsize=(10, 8))
sns.heatmap(correlaciones, annot=True, cmap='coolwarm', fmt=".2f", square=True)
plt.title("📌 Matriz de correlación entre variables")
plt.tight_layout()
plt.show()


In [ ]:
# ---------------------------
# 8. Preparar datos para modelar
# ---------------------------
X = df_modelo[variables_entrada]
y = df_modelo[variable_objetivo]

# Normalizar
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train/Test según año (entrenar con datos de 2016-2017, probar en 2018)
X_train = X[df_modelo['year'] < 2018]
X_test = X[df_modelo['year'] == 2018]
y_train = y[df_modelo['year'] < 2018]
y_test = y[df_modelo['year'] == 2018]

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [ ]:
# ---------------------------
# 9. Entrenar modelo Random Forest
# ---------------------------

# ---------------------------
# 9b. Optimización bayesiana de hiperparámetros para Random Forest
# ---------------------------
# Instalar scikit-optimize si no está instalado
%pip install scikit-optimize

from skopt import BayesSearchCV
from skopt.space import Integer, Real
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit

# Definir el espacio de búsqueda de hiperparámetros
param_space = {
    'n_estimators': Integer(50, 300),
    'max_depth': Integer(3, 20),
    'min_samples_split': Integer(2, 10),
    'min_samples_leaf': Integer(1, 8),
    'max_features': Real(0.5, 1.0, prior='uniform')
}

# Usar solo el conjunto de entrenamiento para la búsqueda
cv = TimeSeriesSplit(n_splits=3)
rf = RandomForestRegressor(random_state=42)
opt = BayesSearchCV(
    rf,
    param_space,
    n_iter=32,
    cv=cv,
    scoring='neg_root_mean_squared_error',
    n_jobs=-1,
    random_state=42,
    verbose=2
)
opt.fit(X_train_scaled, y_train)

print("\nMejores hiperparámetros encontrados:")
print(opt.best_params_)
print(f"Mejor score (neg RMSE): {opt.best_score_:.3f}")

# Usar el mejor modelo encontrado para el resto del flujo
modelo = opt.best_estimator_

In [ ]:
# ---------------------------
# 9c. Comparación de modelos con optimización bayesiana
# ---------------------------
%pip install scikit-optimize xgboost

from skopt import BayesSearchCV
from skopt.space import Integer, Real, Categorical
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import pandas as pd
import numpy as np  # Asegura que np.sqrt esté disponible

modelos = {
    'RandomForest': (RandomForestRegressor(random_state=42), {
        'n_estimators': Integer(50, 300),
        'max_depth': Integer(3, 20),
        'min_samples_split': Integer(2, 10),
        'min_samples_leaf': Integer(1, 8),
        'max_features': Real(0.5, 1.0, prior='uniform')
    }),
    'XGBoost': (XGBRegressor(random_state=42, verbosity=0, n_jobs=-1), {
        'n_estimators': Integer(50, 300),
        'max_depth': Integer(3, 20),
        'learning_rate': Real(0.01, 0.3, prior='log-uniform'),
        'subsample': Real(0.5, 1.0),
        'colsample_bytree': Real(0.5, 1.0)
    }),
    'GradientBoosting': (GradientBoostingRegressor(random_state=42), {
        'n_estimators': Integer(50, 300),
        'max_depth': Integer(3, 20),
        'learning_rate': Real(0.01, 0.3, prior='log-uniform'),
        'subsample': Real(0.5, 1.0)
    }),
    'SVR': (SVR(), {
        'C': Real(0.1, 10.0, prior='log-uniform'),
        'epsilon': Real(0.01, 1.0, prior='log-uniform'),
        'kernel': Categorical(['rbf', 'linear'])
    })
}

cv = TimeSeriesSplit(n_splits=3)
resultados = []
mejores_modelos = {}

for nombre, (modelo, espacio) in modelos.items():
    print(f"\n🔎 Optimizando {nombre}...")
    opt = BayesSearchCV(
        modelo,
        espacio,
        n_iter=24,
        cv=cv,
        scoring='neg_root_mean_squared_error',
        n_jobs=-1,
        random_state=42,
        verbose=0
    )
    opt.fit(X_train_scaled, y_train)
    mejores_modelos[nombre] = opt.best_estimator_
    y_train_pred = opt.best_estimator_.predict(X_train_scaled)
    y_test_pred = opt.best_estimator_.predict(X_test_scaled)
    # Cálculo de RMSE corregido para compatibilidad
    train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
    test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
    train_r2 = r2_score(y_train, y_train_pred)
    test_r2 = r2_score(y_test, y_test_pred)
    resultados.append({
        'Modelo': nombre,
        'Best_Params': opt.best_params_,
        'Train_RMSE': train_rmse,
        'Test_RMSE': test_rmse,
        'Train_R2': train_r2,
        'Test_R2': test_r2,
        'Overfitting_RMSE': train_rmse - test_rmse,
        'Overfitting_R2': train_r2 - test_r2
    })
    print(f"  Mejor RMSE validación: {-opt.best_score_:.3f}")
    print(f"  RMSE train: {train_rmse:.3f} | RMSE test: {test_rmse:.3f}")
    print(f"  R2 train: {train_r2:.3f} | R2 test: {test_r2:.3f}")
    print(f"  Overfitting (RMSE): {train_rmse - test_rmse:.3f}")

# Mostrar resumen comparativo
df_resultados = pd.DataFrame(resultados)
display(df_resultados[['Modelo', 'Train_RMSE', 'Test_RMSE', 'Train_R2', 'Test_R2', 'Overfitting_RMSE', 'Overfitting_R2']])

# Seleccionar el mejor modelo según RMSE test
mejor_nombre = df_resultados.sort_values('Test_RMSE').iloc[0]['Modelo']
modelo = mejores_modelos[mejor_nombre]
print(f"\n🏆 Mejor modelo según RMSE test: {mejor_nombre}")
print(f"Hiperparámetros: {df_resultados.loc[df_resultados['Modelo']==mejor_nombre, 'Best_Params'].values[0]}")

En este paso, el modelo se entrena utilizando la mejor combinación de hiperparámetros encontrada por la búsqueda bayesiana (`BayesSearchCV`). El flujo es el siguiente:

1. **Optimización bayesiana**:  
   Se exploran distintas combinaciones de hiperparámetros del `RandomForestRegressor` (como número de árboles, profundidad máxima, tamaño mínimo de muestras, etc.) usando validación cruzada sobre el conjunto de entrenamiento.  
   El objetivo es minimizar el error cuadrático medio (RMSE) negativo.

2. **Selección del mejor modelo**:  
   Al finalizar la búsqueda, `opt.best_estimator_` contiene el modelo Random Forest entrenado con los mejores hiperparámetros encontrados.

3. **Entrenamiento final**:  
   El modelo óptimo ya está entrenado con los datos de entrenamiento (`X_train_scaled`, `y_train`).  
   No es necesario volver a entrenar el modelo manualmente después de la búsqueda, ya que `BayesSearchCV` lo hace automáticamente.

4. **Uso posterior**:  
   El modelo (`modelo`) se utiliza directamente para predecir y evaluar sobre el conjunto de prueba (2018).

En resumen:  
Aquí ya tienes un modelo Random Forest entrenado y optimizado, listo para hacer predicciones y evaluar su desempeño. ¿Te gustaría una explicación más detallada de los hiperparámetros o del proceso de validación cruzada?

**Entrenamiento del modelo:**

El entrenamiento de los modelos de machine learning se realizó utilizando un enfoque robusto y realista para series de tiempo. Primero, los datos fueron cuidadosamente limpiados y preprocesados, eliminando outliers y registros con valores faltantes en las variables relevantes. Posteriormente, se dividieron en conjuntos de entrenamiento (años 2016-2017) y prueba (año 2018), asegurando así una validación temporal adecuada y evitando el leakage de información del futuro al pasado. Para cada modelo (Random Forest, XGBoost, Gradient Boosting y SVR), se aplicó una optimización bayesiana de hiperparámetros mediante validación cruzada temporal (`TimeSeriesSplit`), buscando minimizar el error cuadrático medio (RMSE) en validación. El mejor modelo de cada tipo se seleccionó automáticamente y se evaluó su desempeño sobre el conjunto de prueba, permitiendo una comparación justa y rigurosa entre algoritmos y evitando el sobreajuste a periodos específicos.

In [ ]:
# ---------------------------
# 10. Evaluación robusta y rigurosa del modelo
# ---------------------------
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, median_absolute_error
import seaborn as sns
import matplotlib.pyplot as plt

# Predicción
y_pred = modelo.predict(X_test_scaled)

# Métricas de desempeño
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)
medae = median_absolute_error(y_test, y_pred)

print("\n🎯 Desempeño global del modelo en datos 2018:")
print(f"   R²: {r2:.3f}")
print(f"   RMSE: {rmse:.3f} toneladas CO₂/ha")
print(f"   MAE: {mae:.3f} toneladas CO₂/ha")
print(f"   MedAE: {medae:.3f} toneladas CO₂/ha")

# Análisis de errores
errores = y_test - y_pred

print("\n📊 Estadísticas de los errores (residuales):")
print(errores.describe().round(3))

# Gráfico de residuales
plt.figure(figsize=(8, 5))
sns.histplot(errores, bins=40, kde=True, color='purple')
plt.title('Distribución de errores (residuales)')
plt.xlabel('Error (NEE observado - NEE predicho) [toneladas CO₂/ha]')
plt.ylabel('Frecuencia')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Gráfico de dispersión predicción vs observación
plt.figure(figsize=(7, 7))
plt.scatter(y_test, y_pred, alpha=0.3, edgecolors='k')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', label='Ideal')
plt.xlabel("NEE observado [toneladas CO₂/ha]")
plt.ylabel("NEE predicho [toneladas CO₂/ha]")
plt.title("Predicción vs Observación (2018)")
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Gráfico de errores vs predicción
plt.figure(figsize=(8, 5))
plt.scatter(y_pred, errores, alpha=0.3, edgecolors='k')
plt.axhline(0, color='red', linestyle='--')
plt.xlabel('NEE predicho [toneladas CO₂/ha]')
plt.ylabel('Error (observado - predicho) [toneladas CO₂/ha]')
plt.title('Error vs NEE predicho')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Análisis temporal de errores
if 'TIMESTAMP_START' in df_modelo.columns:
    test_idx = df_modelo[df_modelo['year'] == 2018].index
    fechas_test = df_modelo.loc[test_idx, 'TIMESTAMP_START']
    plt.figure(figsize=(14, 4))
    plt.plot(fechas_test, errores, '.', alpha=0.5)
    plt.axhline(0, color='red', linestyle='--')
    plt.title('Error temporal (2018)')
    plt.xlabel('Fecha')
    plt.ylabel('Error (observado - predicho) [toneladas CO₂/ha]')
    plt.tight_layout()
    plt.show()

# Evaluación por subgrupos temporales (estaciones)
if 'month' in df_modelo.columns:
    estaciones = {12: 'Verano', 1: 'Verano', 2: 'Verano',
                  3: 'Otoño', 4: 'Otoño', 5: 'Otoño',
                  6: 'Invierno', 7: 'Invierno', 8: 'Invierno',
                  9: 'Primavera', 10: 'Primavera', 11: 'Primavera'}
    meses_test = df_modelo.loc[test_idx, 'month']
    estaciones_test = meses_test.map(estaciones)
    import pandas as pd
    df_eval = pd.DataFrame({'obs': y_test, 'pred': y_pred, 'error': errores, 'estacion': estaciones_test})
    print("\nDesempeño por estación (2018):")
    for est in df_eval['estacion'].unique():
        sub = df_eval[df_eval['estacion'] == est]
        print(f"  {est}: R²={r2_score(sub['obs'], sub['pred']):.3f}, RMSE={np.sqrt(mean_squared_error(sub['obs'], sub['pred'])):.3f} toneladas CO₂/ha, MAE={mean_absolute_error(sub['obs'], sub['pred']):.3f} toneladas CO₂/ha")
    plt.figure(figsize=(8, 5))
    sns.boxplot(x='estacion', y='error', data=df_eval, palette='Set2')
    plt.title('Distribución de errores por estación (2018)')
    plt.xlabel('Estación')
    plt.ylabel('Error (observado - predicho) [toneladas CO₂/ha]')
    plt.tight_layout()
    plt.show()

# Evaluación por cuantiles de NEE observado
cuantiles = pd.qcut(y_test, 4, labels=['Q1 (más bajo)', 'Q2', 'Q3', 'Q4 (más alto)'])
df_eval['cuantil'] = cuantiles
print("\nDesempeño por cuartil de NEE observado (2018):")
for q in df_eval['cuantil'].unique():
    sub = df_eval[df_eval['cuantil'] == q]
    print(f"  {q}: R²={r2_score(sub['obs'], sub['pred']):.3f}, RMSE={np.sqrt(mean_squared_error(sub['obs'], sub['pred'])):.3f} toneladas CO₂/ha, MAE={mean_absolute_error(sub['obs'], sub['pred']):.3f} toneladas CO₂/ha")
plt.figure(figsize=(8, 5))
sns.boxplot(x='cuantil', y='error', data=df_eval, palette='Set3')
plt.title('Distribución de errores por cuartil de NEE observado (2018)')
plt.xlabel('Cuartil de NEE observado')
plt.ylabel('Error (observado - predicho) [toneladas CO₂/ha]')
plt.tight_layout()
plt.show()



ANÁLISIS NEE (EMISIÓN DE DIÓXIDO DE CARBONO)

In [ ]:
# =============================
# Promedios diarios, mensuales y anuales de NEE (base original) y gráficos
# =============================
import matplotlib.pyplot as plt
import seaborn as sns

# Asegurarse de que la columna de fecha es datetime
if 'TIMESTAMP_START' in df.columns:
    df['fecha'] = pd.to_datetime(df['TIMESTAMP_START'])
else:
    df['fecha'] = pd.to_datetime(df['fecha'])

# 1. Promedio diario
nee_diario = df.groupby(df['fecha'].dt.date)['NEE_PI_F'].mean()

# 2. Promedio mensual
nee_mensual = df.groupby(df['fecha'].dt.to_period('M'))['NEE_PI_F'].mean()

# 3. Promedio anual
nee_anual = df.groupby(df['fecha'].dt.year)['NEE_PI_F'].mean()

# Graficar promedios diarios
plt.figure(figsize=(14, 4))
nee_diario.plot()
plt.title('Promedio diario de NEE (base original)')
plt.xlabel('Fecha')
plt.ylabel('NEE (toneladas CO₂/ha)')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Graficar promedios mensuales
plt.figure(figsize=(12, 4))
nee_mensual.plot(marker='o')
plt.title('Promedio mensual de NEE (base original)')
plt.xlabel('Mes')
plt.ylabel('NEE (toneladas CO₂/ha)')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Graficar promedios anuales
plt.figure(figsize=(8, 4))
nee_anual.plot(marker='o', color='green')
plt.title('Promedio anual de NEE (base original)')
plt.xlabel('Año')
plt.ylabel('NEE (toneladas CO₂/ha)')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Mostrar los valores calculados
display(nee_diario.head())
display(nee_mensual.head())
display(nee_anual)

In [ ]:
# =============================
# Contraste NEE observado vs predicho en 2018
# =============================
import matplotlib.pyplot as plt
import pandas as pd

# Asegurarse de que existen las variables necesarias
def get_fechas_test():
    if 'fechas_test' in locals() or 'fechas_test' in globals():
        return fechas_test
    elif 'df_modelo' in locals() or 'df_modelo' in globals():
        test_idx = df_modelo[df_modelo['year'] == 2018].index
        return df_modelo.loc[test_idx, 'TIMESTAMP_START']
    else:
        raise ValueError('No se encontró la variable de fechas para el test.')

fechas_test = get_fechas_test()

# Crear DataFrame de comparación
comparacion = pd.DataFrame({
    'fecha': pd.to_datetime(fechas_test).reset_index(drop=True),
    'NEE_obs': y_test.reset_index(drop=True),
    'NEE_pred': y_pred
})

# Promedios diarios
nee_diario_obs = comparacion.groupby(comparacion['fecha'].dt.date)['NEE_obs'].mean()
nee_diario_pred = comparacion.groupby(comparacion['fecha'].dt.date)['NEE_pred'].mean()

# Promedios mensuales
nee_mensual_obs = comparacion.groupby(comparacion['fecha'].dt.to_period('M'))['NEE_obs'].mean()
nee_mensual_pred = comparacion.groupby(comparacion['fecha'].dt.to_period('M'))['NEE_pred'].mean()

# Promedios anuales
nee_anual_obs = comparacion.groupby(comparacion['fecha'].dt.year)['NEE_obs'].mean()
nee_anual_pred = comparacion.groupby(comparacion['fecha'].dt.year)['NEE_pred'].mean()

# Graficar promedios diarios
plt.figure(figsize=(14, 4))
nee_diario_obs.plot(label='Observado', color='black')
nee_diario_pred.plot(label='Predicho', color='orange')
plt.title('Promedio diario de NEE en 2018: Observado vs Predicho')
plt.xlabel('Fecha')
plt.ylabel('NEE (toneladas CO₂/ha)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Graficar promedios mensuales
plt.figure(figsize=(10, 4))
nee_mensual_obs.plot(marker='o', label='Observado', color='black')
nee_mensual_pred.plot(marker='o', label='Predicho', color='orange')
plt.title('Promedio mensual de NEE en 2018: Observado vs Predicho')
plt.xlabel('Mes')
plt.ylabel('NEE (toneladas CO₂/ha)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Graficar promedios anuales
plt.figure(figsize=(6, 4))
nee_anual_obs.plot(marker='o', label='Observado', color='black')
nee_anual_pred.plot(marker='o', label='Predicho', color='orange')
plt.title('Promedio anual de NEE en 2018: Observado vs Predicho')
plt.xlabel('Año')
plt.ylabel('NEE (toneladas CO₂/ha)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Gráfica de barras para el promedio anual observado vs predicho
plt.figure(figsize=(5, 5))
bar_labels = ['Observado', 'Predicho']
bar_values = [float(nee_anual_obs.values[-1]), float(nee_anual_pred.values[-1])]
plt.bar(bar_labels, bar_values, color=['black', 'orange'])
plt.title('Comparación anual de NEE en 2018')
plt.ylabel('NEE (toneladas CO₂/ha)')
for i, v in enumerate(bar_values):
    plt.text(i, v, f'{v:.2f}', ha='center', va='bottom', fontsize=12)
plt.tight_layout()
plt.show()

# Mostrar diferencias agregadas
print('Diferencia promedio diaria (Obs - Pred):', (nee_diario_obs - nee_diario_pred).mean().round(3))
print('Diferencia promedio mensual (Obs - Pred):', (nee_mensual_obs - nee_mensual_pred).mean().round(3))
print('Diferencia promedio anual (Obs - Pred):', (nee_anual_obs - nee_anual_pred).mean().round(3))

# Mostrar primeros valores
print('\nPrimeros valores diarios:')
display(pd.DataFrame({'Obs': nee_diario_obs, 'Pred': nee_diario_pred}).head())
print('\nValores mensuales:')
display(pd.DataFrame({'Obs': nee_mensual_obs, 'Pred': nee_mensual_pred}))
print('\nValores anuales:')
display(pd.DataFrame({'Obs': nee_anual_obs, 'Pred': nee_anual_pred}))

ESTIMAR EL MODELO ML SOBRE CALDAS - COLOMBIA 

In [ ]:
# =============================
# Descarga, procesamiento y predicción sobre Caldas, Colombia
# =============================

# NOTA: Este es un flujo ejemplo. Para producción, adapta credenciales y parámetros según tu acceso a APIs.

# 1. Descarga de NDVI/PRI desde Sentinel-2 (usando Google Earth Engine o SentinelHub)
# Aquí se muestra un ejemplo usando geemap y GEE para NDVI. Para producción, usa tu cuenta de GEE o descarga manual.

import geemap
import ee
import pandas as pd
import numpy as np
from datetime import datetime

# Inicializar GEE (requiere autenticación la primera vez)
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

# Definir región de Caldas (ejemplo: bounding box)
caldas_bbox = ee.Geometry.BBox(-76.2, 4.7, -75.0, 5.8)  # Ajusta según necesidad

# Definir periodo de interés
fecha_inicio = '2018-01-01'
fecha_fin = '2018-12-31'

# Cargar colección Sentinel-2 y calcular NDVI
s2 = ee.ImageCollection('COPERNICUS/S2_SR') \
    .filterBounds(caldas_bbox) \
    .filterDate(fecha_inicio, fecha_fin) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))

def add_ndvi(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return image.addBands(ndvi)

s2_ndvi = s2.map(add_ndvi)

# Reducir a valores medios diarios sobre la región
ndvi_diario = s2_ndvi.select('NDVI').mean().reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry=caldas_bbox,
    scale=20,
    maxPixels=1e8
)
ndvi_valor = ndvi_diario.get('NDVI').getInfo()

# 2. Descarga de meteorología ERA5-Land (ejemplo usando archivo NetCDF ya descargado)
import xarray as xr

# Suponiendo que tienes el archivo NetCDF descargado en 'datos_netcdf/radiacion_solar_era5_2024_01.nc'
# Para producción, automatiza la descarga con CDS API si es necesario
nc_path = 'datos_netcdf/radiacion_solar_era5_2024_01.nc'
ds = xr.open_dataset(nc_path)

# Extraer variable de radiación solar y promedio sobre la región
# Ajusta lat/lon según Caldas
lat_min, lat_max = 4.7, 5.8
lon_min, lon_max = -76.2, -75.0

ds_caldas = ds.sel(latitude=slice(lat_max, lat_min), longitude=slice(lon_min, lon_max))

# Promedio temporal y espacial
rad_solar = ds_caldas['ssrd'].mean(dim=['latitude', 'longitude']).to_dataframe().reset_index()

# 3. Unir datos de NDVI y meteorología (ejemplo simple)
# Aquí solo se muestra NDVI y radiación solar, pero debes agregar todas las variables requeridas por el modelo

df_pred = pd.DataFrame({
    'NDVI': [ndvi_valor],
    'SW_IN_1_1_1': [rad_solar['ssrd'].mean() / (24*3600)],  # Convertir J/m2 a W/m2 si es necesario
    # Agrega aquí el resto de variables meteorológicas y de teledetección
    # Usa valores promedio o temporales según tu resolución
})

# 4. Preprocesar y normalizar igual que en el entrenamiento
X_pred = df_pred[variables_entrada]
X_pred_scaled = scaler.transform(X_pred)

# 5. Predecir NEE para Caldas
nee_pred_caldas = modelo.predict(X_pred_scaled)
print(f"Predicción de NEE para Caldas (promedio periodo): {nee_pred_caldas[0]:.3f} toneladas CO₂/ha")

# NOTA: Para predicción diaria/mensual, repite el proceso para cada fecha y une los resultados.